In [1]:
import pandas as pd
import numpy as np

**Import Hauptdaten**

In [2]:
df = pd.read_excel('20210607 Covid-19-Daten.xlsx', sheet_name='2.1 Daten Gemeinden')

In [3]:
df

,Bezirk,Gemeinde,Einwohnerzahl (Stand 30.06.2020),Gesamtzahl seit Start Contact Tracing (11.05.2020),Anz Infektionen Tage x-6 bis x (kuratiert),Anz Infektionen Tage x-13 bis x-7 (kuratiert)
0,Aarau,Aarau,21814,1471,0-3,7
1,Aarau,Suhr,10512,864,0-3,7
2,Aarau,Oberentfelden,8534,742,4,0-3
3,Aarau,Buchs (AG),8018,645,0-3,0-3
4,Aarau,Gränichen,8039,508,6,0-3
...,...,...,...,...,...,...
205,Zurzach,Rekingen (AG),940,57,0-3,0-3
206,Zurzach,Schneisingen,1485,57,0-3,0-3
207,Zurzach,Siglistorf,649,51,0-3,0-3
208,Zurzach,Rietheim,722,49,0-3,0-3


In [4]:
df.columns = ['bezirk', 'gemeinde', 'einwohner', 'fälle_total', 'letzte Woche', 'vorletzte Woche']

In [5]:
df['gemeinde'] = df['gemeinde'].str.replace(' \(AG\)', '')

In [6]:
df.set_index('gemeinde', inplace=True)

**Berechnung Fälle pro Hundert Einwohner:innen**

In [7]:
df['fälle_total'] = df['fälle_total'].replace('Keine Publikation', np.nan)

In [8]:
df['fälle_pro_hundert'] = (df['fälle_total'] / df['einwohner']) * 100

**Import Wappen + Merge**

In [9]:
df_wappen = pd.read_csv('../../Vorlagen/bfs-nummer_gemeinde_wappen_2020.csv')
df_wappen['Ort'] = df_wappen['Ort'].str.replace(' \(AG\)', '')
df_wappen.set_index('Ort', inplace=True)

In [10]:
df = pd.merge(df, df_wappen, left_index=True, right_index=True, how='left')

In [11]:
df.reset_index(inplace=True)
df.set_index('BFS-Nummer', inplace=True)

**Import Einkommen + Merge**

In [12]:
col_names = ['BFS-Nummer', 'Gemeindename', 'Steuerbares Einkommen total', 'Steuerbares Einkommen pro Kopf']
df_einkommen = pd.read_excel('Einkommen_AG_2017.xlsx', names=col_names, skiprows=5)
del df_einkommen['Steuerbares Einkommen total']
df_einkommen = df_einkommen.loc[:2254].copy()
df_einkommen['BFS-Nummer'] = df_einkommen['BFS-Nummer'].astype(int)
df_einkommen['Steuerbares Einkommen pro Kopf'] = df_einkommen['Steuerbares Einkommen pro Kopf'].astype(float)
df_einkommen.set_index('BFS-Nummer', inplace=True)
df_einkommen = df_einkommen[(df_einkommen.index >= 4001) & (df_einkommen.index <= 4323)]

In [13]:
df = pd.merge(df, df_einkommen, left_index=True, right_index=True, how='left')

In [14]:
df_vermögen = pd.read_excel('gemeindefinanzen_2019.xlsx', sheet_name='T12', skiprows=3)
df_vermögen = df_vermögen[['BFSNR', 'Gemeinde', 'einwohner', 'nettovermögen_1000_franken']].copy()
df_vermögen = df_vermögen.loc[:222]
df_vermögen['vermögen_pro_tausend'] = ((df_vermögen['nettovermögen_1000_franken'] * 1000) / df_vermögen['einwohner']) * 1000
df_vermögen.set_index('BFSNR', inplace=True)

In [15]:
df = pd.merge(df, df_vermögen[['vermögen_pro_tausend']], left_index=True, right_index=True, how='left')

**Hauptexport Datawrapper-Karte**

In [16]:
df[['gemeinde', 'einwohner', 'fälle_total', 'fälle_pro_hundert', 'Wappen']].to_csv('Exporte/fälle_pro_hundert.csv')

**Diese zehn Gemeinden haben am wenigsten Fälle**

In [17]:
#pro 100 EW
df.sort_values(by='fälle_pro_hundert', ascending=True).head(10).round(1)#[['gemeinde', 'fälle_pro_hundert']].to_csv('Exporte/wenigste_fälle.csv')

,gemeinde,bezirk,einwohner,fälle_total,letzte Woche,vorletzte Woche,fälle_pro_hundert,Wappen,Gemeindename,Steuerbares Einkommen pro Kopf,vermögen_pro_tausend
BFS-Nummer,,,,,,,,,,,
4203,Möriken-Wildegg,Lenzburg,4491,85.0,0-3,0-3,1.9,http://i.imgur.com/0EUBHOL.png,Möriken-Wildegg,79443.3,-2376090.2
4184,Mettauertal,Laufenburg,2046,53.0,0-3,0-3,2.6,http://i.imgur.com/gALhRfi.png,Mettauertal,68943.4,760749.0
4191,Ammerswil,Lenzburg,721,19.0,0-3,0-3,2.6,http://i.imgur.com/afyGrTN.png,Ammerswil,76925.9,-2392266.8
4134,Dürrenäsch,Kulm,1313,46.0,0-3,0-3,3.5,http://i.imgur.com/yvxFBnD.png,Dürrenäsch,69085.0,-5395767.3
4173,Oberhof,Laufenburg,589,21.0,0-3,0-3,3.6,http://i.imgur.com/z0ORwr7.png,Oberhof,68539.1,-593949.4
4124,Bözberg,Brugg,1650,61.0,0-3,0-3,3.7,http://i.imgur.com/Z82TA8D.png,Bözberg,75656.2,-4273907.6
4145,Teufenthal,Kulm,1669,62.0,0-3,0-3,3.7,http://i.imgur.com/NvOAkJh.png,Teufenthal (AG),59972.2,2498537.4
4091,Auenstein,Brugg,1615,60.0,0-3,0-3,3.7,http://i.imgur.com/uT769MY.png,Auenstein,84652.1,-1535789.0
4318,Schneisingen,Zurzach,1485,57.0,0-3,0-3,3.8,http://i.imgur.com/VRUk88K.png,Schneisingen,76553.2,1737727.4


In [18]:
#Fälle total
df.sort_values(by='fälle_total').head(10).round(2)

,gemeinde,bezirk,einwohner,fälle_total,letzte Woche,vorletzte Woche,fälle_pro_hundert,Wappen,Gemeindename,Steuerbares Einkommen pro Kopf,vermögen_pro_tausend
BFS-Nummer,,,,,,,,,,,
4191,Ammerswil,Lenzburg,721,19.0,0-3,0-3,2.64,http://i.imgur.com/afyGrTN.png,Ammerswil,76925.93,-2392266.81
4173,Oberhof,Laufenburg,589,21.0,0-3,0-3,3.57,http://i.imgur.com/z0ORwr7.png,Oberhof,68539.09,-593949.41
4306,Fisibach,Zurzach,526,28.0,0-3,0-3,5.32,http://i.imgur.com/yTT7fDF.png,Fisibach,71289.22,-5032356.63
4004,Densbüren,Aarau,732,32.0,0-3,0-3,4.37,http://i.imgur.com/0PxOfpX.png,Densbüren,67091.47,-257226.71
4227,Bettwil,Muri,662,33.0,0-3,0-3,4.98,http://i.imgur.com/OaApds5.png,Bettwil,66944.48,-1761451.05
4226,Besenbüren,Muri,627,34.0,0-3,0-3,5.42,http://i.imgur.com/yiA3B1k.png,Besenbüren,76356.63,-4131788.19
4179,Ueken,Laufenburg,904,35.0,0-3,0-3,3.87,http://i.imgur.com/BVhtXoi.png,Ueken,72593.42,-636271.15
4117,Thalheim,Brugg,818,36.0,0-3,0-3,4.40,http://i.imgur.com/esL6zNk.png,Thalheim (AG),70882.73,1403815.21
4251,Hellikon,Rheinfelden,783,40.0,0-3,0-3,5.11,http://i.imgur.com/J5RSMbU.png,Hellikon,62478.86,2306557.26


**Diese zehn Gemeinden haben am meisten Fälle**

In [19]:
#pro 100 EW
df.sort_values(by='fälle_pro_hundert', ascending=False).head(10).round(1)#[['gemeinde', 'fälle_pro_hundert']].to_csv('Exporte/meiste_fälle.csv')


,gemeinde,bezirk,einwohner,fälle_total,letzte Woche,vorletzte Woche,fälle_pro_hundert,Wappen,Gemeindename,Steuerbares Einkommen pro Kopf,vermögen_pro_tausend
BFS-Nummer,,,,,,,,,,,
4042,Turgi,Baden,2966,340.0,0-3,0-3,11.5,http://i.imgur.com/PzCWLUz.png,Turgi,66957.6,-2392503.3
4027,Fislisbach,Baden,5606,555.0,0-3,0-3,9.9,http://i.imgur.com/Nlv59so.png,Fislisbach,70767.2,820408.7
4040,Spreitenbach,Baden,12146,1189.0,0-3,5,9.8,http://i.imgur.com/rTrCwxh.png,Spreitenbach,61254.9,225268.3
4072,Niederwil,Bremgarten,2810,263.0,0-3,0-3,9.4,http://i.imgur.com/W6rKm9n.png,Niederwil (AG),73477.7,-382268.7
4310,Koblenz,Zurzach,1690,153.0,0-3,0-3,9.1,http://i.imgur.com/NUihGv5.png,Koblenz,63660.9,343401.9
4065,Dottikon,Bremgarten,3904,341.0,0-3,0-3,8.7,http://i.imgur.com/0mUHtpT.png,Dottikon,65475.7,-5657235.4
4010,Oberentfelden,Aarau,8534,742.0,4,0-3,8.7,http://i.imgur.com/eNr44Vb.png,Oberentfelden,65319.0,-1727511.9
4034,Neuenhof,Baden,8989,779.0,0-3,0-3,8.7,http://i.imgur.com/yrqjhgc.png,Neuenhof,60523.5,3305205.4
4170,Laufenburg,Laufenburg,3668,313.0,0-3,0-3,8.5,http://i.imgur.com/0q2DzDX.png,Laufenburg,71166.6,2921767.3


In [20]:
#Fälle total
df.sort_values(by='fälle_total', ascending=False)

,gemeinde,bezirk,einwohner,fälle_total,letzte Woche,vorletzte Woche,fälle_pro_hundert,Wappen,Gemeindename,Steuerbares Einkommen pro Kopf,vermögen_pro_tausend
BFS-Nummer,,,,,,,,,,,
4045,Wettingen,Baden,21106,1594.0,8,14,7.552355,http://i.imgur.com/bZt880G.png,Wettingen,77000.823211,5.358961e+06
4001,Aarau,Aarau,21814,1471.0,0-3,7,6.743376,http://i.imgur.com/JWqWyRS.png,Aarau,74765.250929,-6.269019e+06
4021,Baden,Baden,19531,1328.0,4,9,6.799447,http://i.imgur.com/in3AmXY.png,Baden,83135.741167,2.612405e+05
4040,Spreitenbach,Baden,12146,1189.0,0-3,5,9.789231,http://i.imgur.com/rTrCwxh.png,Spreitenbach,61254.940801,2.252683e+05
4082,Wohlen,Bremgarten,16661,1189.0,4,0-3,7.136426,http://i.imgur.com/toO1BXR.png,Wohlen (AG),68021.741778,3.598549e+05
...,...,...,...,...,...,...,...,...,...,...,...
4314,Mellikon,Zurzach,227,NaN,--,--,NaN,http://i.imgur.com/nxzSlN6.png,Mellikon,71234.042553,-1.897017e+06
4301,Baldingen,Zurzach,273,NaN,--,--,NaN,http://i.imgur.com/BEwN8fD.png,Baldingen,73464.102564,1.292088e+05
4317,Rümikon,Zurzach,333,NaN,--,--,NaN,http://i.imgur.com/yB4U0hv.png,Rümikon,62072.222222,-4.194815e+06


**Zusammenhang Corona-Fälle und Einkommen?**

In [22]:
df[['gemeinde', 'fälle_pro_hundert', 'Steuerbares Einkommen pro Kopf']].to_csv('Exporte/fälle_einkommen.csv')

In [23]:
df[['gemeinde', 'fälle_pro_hundert', 'vermögen_pro_tausend']].to_csv('Exporte/fälle_gemeindevermögen.csv')

**Aufteilung nach Bezirken**

In [25]:
df_bezirke = pd.DataFrame(df.groupby('bezirk')['fälle_pro_hundert'].mean().sort_values())

In [26]:
df_bezirke.reset_index(inplace=True)

In [27]:
df_bezirke = df_bezirke.sort_values(by='fälle_pro_hundert', ascending=False)

In [28]:
df_bezirke.to_csv('Exporte/bezirke.csv', index=False)

**Rank und Abweichung vom Mittelwert hinzufügen**

In [29]:
df['Abweichung Mittelwert'] = df['fälle_pro_hundert'] - df['fälle_pro_hundert'].mean()

In [30]:
df['rank'] = df['fälle_pro_hundert'].rank()

In [37]:
df[df['fälle_pro_hundert'].notnull()][['rank', 'gemeinde', 'fälle_pro_hundert', 'Abweichung Mittelwert', 'fälle_total']].sort_values(by='fälle_pro_hundert', ascending=False).to_csv('Exporte/alle_gemeinden_abweichung_mittelwert.csv')

**Aktuelle Daten**

In [38]:
df = df.replace('--', np.nan)

In [40]:
df[['gemeinde', 'letzte Woche', 'vorletzte Woche']].to_csv('Exporte/aktuelle_zahlen.csv')